<a href="https://colab.research.google.com/github/Vaipik/GoIT-PyDS7/blob/main/hw12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np


In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [3]:
train_images = np.dstack([train_images] * 3)
test_images = np.dstack([test_images] * 3)

In [4]:
train_images = train_images.reshape(-1, 28, 28, 3)
test_images = test_images.reshape(-1, 28, 28, 3)

In [5]:
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
train_images = np.asarray(
    [img_to_array(array_to_img(img, scale=False).resize((48,48))) for img in train_images])
test_images = np.asarray([img_to_array(array_to_img(img, scale=False).resize((48,48))) for img in test_images])



In [6]:
train_images = train_images.astype("float32") / 255
test_images = test_images.astype("float32") / 255

In [7]:
from tensorflow.keras.applications.vgg16 import VGG16

In [8]:
vgg16_base = VGG16(weights="imagenet", include_top=False, input_shape=(48, 48, 3))
vgg16_base.trainable = True
set_trainable = False

layer_train_lst = ['block5_conv1', 'block5_conv2', 'block5_conv3']

for layer in vgg16_base.layers:
    if layer.name in layer_train_lst:
        set_trainable = True
    elif set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [9]:
model = tf.keras.Sequential([
    vgg16_base,    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation="sigmoid", kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dense(10, activation="softmax", name='out_layer')
])

In [10]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
    )

In [11]:
model.fit(
    train_images,
    train_labels,
    validation_split=0.1,
    batch_size=256,
    epochs=30
    )

Epoch 1/30
211/211 [==============================] - 36s 98ms/step - loss: 1.2057 - accuracy: 0.7470 - val_loss: 0.7448 - val_accuracy: 0.8317
Epoch 2/30
211/211 [==============================] - 18s 86ms/step - loss: 0.6345 - accuracy: 0.8490 - val_loss: 0.5634 - val_accuracy: 0.8558
Epoch 3/30
211/211 [==============================] - 18s 86ms/step - loss: 0.4958 - accuracy: 0.8668 - val_loss: 0.5165 - val_accuracy: 0.8533
Epoch 4/30
211/211 [==============================] - 18s 85ms/step - loss: 0.4155 - accuracy: 0.8806 - val_loss: 0.4301 - val_accuracy: 0.8703
Epoch 5/30
211/211 [==============================] - 18s 85ms/step - loss: 0.3637 - accuracy: 0.8910 - val_loss: 0.3883 - val_accuracy: 0.8837
Epoch 6/30
211/211 [==============================] - 17s 79ms/step - loss: 0.3323 - accuracy: 0.8983 - val_loss: 0.3660 - val_accuracy: 0.8797
Epoch 7/30
211/211 [==============================] - 18s 85ms/step - loss: 0.3085 - accuracy: 0.9043 - val_loss: 0.3663 - val_accuracy:

In [12]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('Test accuracy:', test_acc)

313/313 - 4s - loss: 0.4385 - accuracy: 0.8912 - 4s/epoch - 14ms/step
Test accuracy: 0.8912000060081482
